In [8]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [9]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2025-03-09 19:18:20--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.2.33, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip.4’

book-crossings.zip. 100%[===================>]  24.88M  --.-KB/s    in 0.1s    

2025-03-09 19:18:20 (203 MB/s) - ‘book-crossings.zip.4’ saved [26085508/26085508]

Archive:  book-crossings.zip
replace BX-Book-Ratings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: BX-Book-Ratings.csv     
replace BX-Books.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: BX-Books.csv            
replace BX-Users.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: BX-Users.csv            


In [13]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [11]:
# add your code here - consider creating a new cell for each section of code
df_ratings.head()
df_ratings.shape



(1149780, 3)

In [12]:
df_ratings = df_ratings[(df_ratings['rating'] >= 1) & (df_ratings['rating'] <= 10)]

# Verify the shape of the new data
print("Shape of cleaned ratings data (after removing 0.0 ratings):", df_ratings.shape)
df_ratings.head()
df_books.head()

Shape of cleaned ratings data (after removing 0.0 ratings): (433671, 3)


,isbn,title,author
0,0195153448,Classical Mythology,Mark P. O. Morford
1,0002005018,Clara Callan,Richard Bruce Wright
2,0060973129,Decision in Normandy,Carlo D'Este
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata
4,0393045218,The Mummies of Urumchi,E. J. W. Barber


In [14]:
# 1. Remove users with less than 200 ratings
user_counts = df_ratings['user'].value_counts()
active_users = user_counts[user_counts >= 200].index
df_ratings = df_ratings[df_ratings['user'].isin(active_users)]

# 2. Remove books with less than 100 ratings
book_counts = df_ratings['isbn'].value_counts()
popular_books = book_counts[book_counts >= 100].index
df_ratings = df_ratings[df_ratings['isbn'].isin(popular_books)]

# Verify the shape of cleaned data
print("Shape of cleaned ratings data:", df_ratings.shape)

Shape of cleaned ratings data: (13793, 3)


In [15]:
df_merged = pd.merge(df_ratings,df_books, on='isbn')
book_user_matrix=df_merged.pivot_table(index='title',columns='user',values='rating')
book_user_matrix = book_user_matrix.fillna(0)

print("Shape of the matrix:", book_user_matrix.shape)

Shape of the matrix: (99, 857)


In [16]:
book_matrix = book_user_matrix.values
model = NearestNeighbors(metric='cosine', algorithm='brute')
model.fit(book_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [17]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
  book_index = book_user_matrix.index.get_loc(book)
  distances,indices = model.kneighbors(book_matrix[book_index].reshape(1,-1), n_neighbors=6)
  recommended_books = []
  recommended_books.append(book_user_matrix.index[indices[0][0]])
  recommended_books.append([])
  for i in range(1, 6):
        recommended_books[1].append([book_user_matrix.index[indices[0][i]], distances[0][i]])


  return recommended_books
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

["Where the Heart Is (Oprah's Book Club (Paperback))", [['The Lovely Bones: A Novel', 0.7234864], ["The Pilot's Wife : A Novel", 0.8192678], ['The Joy Luck Club', 0.81986046], ['The Notebook', 0.8236683], ['Bel Canto: A Novel', 0.8247875]]]


In [18]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [['The Lovely Bones: A Novel', 0.7234864], ["The Pilot's Wife : A Novel", 0.8192678], ['The Joy Luck Club', 0.81986046], ['The Notebook', 0.8236683], ['Bel Canto: A Novel', 0.8247875]]]
You haven't passed yet. Keep trying!
